# Composite pattern (again)

> Dealing with more complex scenarios

Let's try a more complex scenario now: we're going to create a neural network.

Let's begin by defining a `Neuron` class. A neuron is characterized by their connections, so our neurons will have 2 lists:
* inputs (neurons that will connect to our neuron)
* outputs (neurons that our neuron will connect to)

We will also create a `connect_to` method that will be used to connect our neuron to another one.

In [1]:
class Neuron:
    def __init__(self, name):
        self.name = name
        self.inputs = []
        self.outputs = []

    def __str__(self):
        return f'{self.name}, {len(self.inputs)} inputs, {len(self.outputs)} outputs'

    def connect_to(self, other):
        self.outputs.append(other)
        other.inputs.append(self)

This code will work fine, but maybe we want to work with groups of neurons at a time. We could create some sort of `NeuronLayer` that will contain a list of `Neuron`s:

In [2]:
class NeuronLayer(list):
    def __init__(self, name, count):
        super().__init__()
        self.name = name
        for x in range(0, count):
            self.append(Neuron(f'{name}-{x}')) # we name each neuron with the name of the layer and its position in the list

    def __str__(self):
        return f'{self.name} with {len(self)} neurons'

However, we now have a problem: We can connect neurons to other neurons, but we don't have a way to connect a neuron to a layer, a layer to a neuron or a layer to a layer. We could implement methods for each scenario, but what we want is just a single method that will take care of all of them. How can we write this method?

For example, we could move the `Neuron.connect_to` method outside the class and use it as a separate method that could iterate both the neurons in a layer and the neurons in a neuron:

In [3]:
class Neuron:
    def __init__(self, name):
        self.name = name
        self.inputs = []
        self.outputs = []

    def __str__(self):
        return f'{self.name}, {len(self.inputs)} inputs, {len(self.outputs)} outputs'
    
    # we move the connect_to method outside the class

def connect_to(self, other): # self doesn't refer to an instance; we just keep the names from the original method
    if self == other:
        return

    for s in self:
        for o in other:
            s.outputs.append(o)
            o.inputs.append(s)

With this method in place, we could now imbue it into both `Neuron` and `NeuronLayer` and use it from those classes. Let's try it out:

In [4]:
Neuron.connect_to = connect_to
NeuronLayer.connect_to = connect_to

neuron1 = Neuron('n1')
neuron2 = Neuron('n2')
layer1 = NeuronLayer('L1', 3)
layer2 = NeuronLayer('L2', 4)

neuron1.connect_to(neuron2)
neuron1.connect_to(layer1)
layer1.connect_to(neuron2)
layer1.connect_to(layer2)

print(neuron1)
print(neuron2)
print(layer1)
print(layer2)

TypeError: 'Neuron' object is not iterable

Oops, that didn't work. The line `for s in self` inside `connect_to` seems to be causing the error; the reason is that this specific line is trying to iterate through and object which happens to be a scalar rather than iterable (the `Neuron`).

But his is very easy to solve: we simply implement `__iter__` for the class and make it yield itself.

In [6]:
class Neuron():
    def __init__(self, name):
        self.name = name
        self.inputs = []
        self.outputs = []

    def __str__(self):
        return f'{self.name}, {len(self.inputs)} inputs, {len(self.outputs)} outputs'
    
    # let's make our neuron iterable
    def __iter__(self):
        yield self

Let's run the same code as before:

In [7]:
Neuron.connect_to = connect_to
NeuronLayer.connect_to = connect_to

neuron1 = Neuron('n1')
neuron2 = Neuron('n2')
layer1 = NeuronLayer('L1', 3)
layer2 = NeuronLayer('L2', 4)

neuron1.connect_to(neuron2)
neuron1.connect_to(layer1)
layer1.connect_to(neuron2)
layer1.connect_to(layer2)

print(neuron1)
print(neuron2)
print(layer1)
print(layer2)

n1, 0 inputs, 4 outputs
n2, 4 inputs, 0 outputs
L1 with 3 neurons
L2 with 4 neurons


It now works!

We could leave the solution as it currently stands, but `connect_to` is now just out there, freestanding, which is kind of ugly.

We could create a base class that would contain `connect_to`, and we can make it both iterable and abstract so that both `Neuron` and `NeuronLayer` can inherit from it:

In [8]:
from abc import ABC
from collections.abc import Iterable

class Connectable(Iterable, ABC):
    def connect_to(self, other):
        if self == other:
            return

        for s in self:
            for o in other:
                s.outputs.append(o)
                o.inputs.append(s)

Now let's modify both `Neuron` and `NeuronLayer` so that they both inherit from `Connectable`.

In [9]:
class Neuron(Connectable):
    def __init__(self, name):
        self.name = name
        self.inputs = []
        self.outputs = []

    def __str__(self):
        return f'{self.name}, {len(self.inputs)} inputs, {len(self.outputs)} outputs'
    
    # Check note below
    def __iter__(self):
        yield self

class NeuronLayer(list, Connectable):
    def __init__(self, name, count):
        super().__init__()
        self.name = name
        for x in range(0, count):
            self.append(Neuron(f'{name}-{x}'))

    def __str__(self):
        return f'{self.name} with {len(self)} neurons'

> `Connectable` is an iterable abstract class, and as such, any class that implements it must be iterable. `Neuron` still needs to implement `__iter__`, because otherwise when we run the code, the interpreter will complain about it. `NeuronLayer` inherits from `list` which is already iterable, so there is no need for it.

Now both `Neuron` and `NeuronLayer` have inherited `connect_to`. Let's test it:

In [10]:
neuron1 = Neuron('n1')
neuron2 = Neuron('n2')
layer1 = NeuronLayer('L1', 3)
layer2 = NeuronLayer('L2', 4)

neuron1.connect_to(neuron2)
neuron1.connect_to(layer1)
layer1.connect_to(neuron2)
layer1.connect_to(layer2)

print(neuron1)
print(neuron2)
print(layer1)
print(layer2)

n1, 0 inputs, 4 outputs
n2, 4 inputs, 0 outputs
L1 with 3 neurons
L2 with 4 neurons


As you can see, we get the same output but we've made the syntax a bit nicer.

In conclusion: we've implemented the composite pattern by having our `Neuron` class behave as a collection even though it's a simple scalar element, and we've done this by implementing `__iter__` and have it yielding itself whenever somebody asks for the iteration.